<img src="../static/logopython.png" alt="Logo Python" style="width: 300px; display: inline"/>
<img src="../static/deimoslogo.png" alt="Logo Deimos" style="width: 300px; display: inline"/>

# Clase 5: Metaprogramación en Python

En la metaprogramación, escribimos <strong>código que trabaja con código</strong>. Algo para lo que Python 3 está <strong>especialmente bien preparado</strong>

Veamos a continuación para qué nos puede interesar escribir código que trabaja con código.

## ¿Qué es eso de la metaprogramación?

Como ya hemos dicho, la metaprogramación consiste en escribir código que va a trabajar con el código ya escrito, incluso consigo mismo.

Al ser Python un lenguaje dinámico, no podemos saber las interioridades de los componentes con los que estemos trabajando, salvo que los inspeccionemos en tiempo de ejecución. Por ejemplo

In [3]:
# Añadimos atributos a una instancia de una clase, una vez creada
class Empleado(object):
    pass

jorge = Empleado()
jorge.guapo = "Por supuesto"

print("Es Jorge guapo: {}".format(jorge.guapo))

Es Jorge guapo: Por supuesto


En la manera más básica de metaprogramación, podemos manipular los atributos de una clase, mediante el trio de funciones [hasattr](https://docs.python.org/3/library/functions.html#hasattr), [getattr](https://docs.python.org/3/library/functions.html#getattr), [setattr](https://docs.python.org/3/library/functions.html#setattr)

In [6]:
# Probando hasattr, getattr, setattr
print(hasattr(jorge, 'guapo'))
print(getattr(jorge, 'guapo'))

print(hasattr(jorge, 'listo'))
setattr(jorge, 'listo', 'Más que las monas')
print(getattr(jorge, 'listo'))

True
Por supuesto
False
Más que las monas


TODO: Meter la parte de introspección del capítulo 3 del libro. Es muy cortito y habla de docstrings, tipos, etc

## ¿Y para qué me vale esto?

La metaprogamación es especialmente útil cuando:

* Construyes herramientas para otros programadores (ej: un IDE)
* Quieres añadir ciertos _extras_ a tu código, para hacerlo más seguro o más fácilmente depurable
* Quieres aprender cómo funciona realmente el lenguaje por dentro... y manipularlo
* ...

Es posible que estos argumentos no te parezcan suficientes. No hay problema. [Tim Peters](http://c2.com/cgi/wiki?TimPeters) dijo esto acerca de las metaclases (uno de los componentes fundamentales de la metaprogramación):

_[Metaclasses] are deeper magic than 99% of
users should ever worry about. If you
wonder whether you need them, you
don't (the people who actually need them
know with certainty that they need them,
and don't need an explanation about why)._

Así que, si no te ha surgido la necesidad de usar metaprogramación, no te preocupes ;-). Nosotros, al menos, vamos a ver un ejemplo de uso

## Caso de uso: depuración de código

¿Cómo depuramos nuestro código? Una primera aproximación (pobre) es... usando print

In [7]:
# Así es como depuran los muggles
def add(x, y):
    print('Dentro de la función add')
    return x + y

add(2, 2)

Dentro de la función add


4

Precioso. ¿Y si tenemos más funciones?

In [2]:
# La cosa se complica...
def add(x, y):
    print('add')
    return x + y

def sub(x, y):
    print('sub')
    return x - y

def mul(x, y):
    print('mul')
    return x * y

def div(x, y):
    print('div')
    return x / y

add(2, 2)
sub(8, 3)
mul(5,6)
div(16, 2)

add
sub
mul
div


8.0

Esto no es nada cómodo. Nos gustaría poder crear un *depurador* como elemento externo, y aplicarle ese elemento a todas las funciones que nos interese. Estamos hablando de los [decorators](https://wiki.python.org/moin/PythonDecorators)

<div class="alert alert-info">Los decorators no deberían ser algo ajeno para ti. Los vimos en la clase de orientación a objetos</div>

Un decorador es una función que actúa como _wrapper_ de otra. El _wrapper_ actúa como la función original pero añade comportamiento adicional. En nuestro caso, lo que queremos es que añada información de debug

In [3]:
from functools import wraps, partial

def debug(func):
    '''
    A simple debugging decorator
    '''
    msg = func.__qualname__
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(msg)
        return func(*args, **kwargs)
    return wrapper

Vamos ahora a redefinir nuestras funciones, aplicando el _wrapper_

In [6]:
@debug
def add(x, y):
    return x + y
@debug
def sub(x, y):
    return x - y
@debug
def mul(x, y):
    return x * y
@debug
def div(x, y):
    return x / y

add(2, 2)
sub(8, 3)
mul(5,6)
div(16, 2)

add
sub
mul
div


8.0

Con este cambio:

* Hemos accedido a la firma de una función (su nombre) para sacar información por pantalla (reflexión)
* Hemos aislado el código de debug en una función aparte, que puede ser aplicada a otras (separación de responsabilidades)

Eso hace feliz a Guido

<img src="https://www.soldierx.com/system/files/hdb/guido-van-rossum.jpg">

Una posible mejora sería utilizar el módulo de logger, que funciona de una manera muy similar al de Java

In [8]:
from functools import wraps
import logging

def debug(func):
    log = logging.getLogger(func.__module__)
    msg = func.__qualname__
    @wraps(func)
    def wrapper(*args, **kwargs):
        log.debug(msg)
        return func(*args, **kwargs)
    return wrapper

Otra adicional sería desactivar el mensaje de DEBUG usando una variable de entorno

In [10]:
from functools import wraps
import logging
import os

def debug(func):
    if 'DEBUG' not in os.environ:
        return func
    log = logging.getLogger(func.__module__)
    msg = func.__qualname__
    @wraps(func)
    def wrapper(*args, **kwargs):
        log.debug(msg)
        return func(*args, **kwargs)
    return wrapper

Imaginemos ahora que queremos añadir un prefijo a todos nuestros mensajes de depuración, y que ese prefijo sea dinámico. Es decir, que se lo podamos pasar como argumento al _wrapper_

In [11]:
def debugargs(prefix=''):
    '''
    A debugging decorator that takes arguments
    '''
    def decorate(func):
        msg = prefix + func.__qualname__
        @wraps(func)
        def wrapper(*args, **kwargs):
            print(msg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

In [13]:
@debugargs(prefix="------")
def add(x, y):
    return x + y
@debugargs(prefix="------")
def sub(x, y):
    return x - y
@debugargs(prefix="------")
def mul(x, y):
    return x * y
@debugargs(prefix="------")
def div(x, y):
    return x / y


add(2, 2)
sub(8, 3)
mul(5,6)
div(16, 2)

------add
------sub
------mul
------div


8.0

Vale, eso está muy bien, pero ahora tengo dos wrappers: uno sin argumentos, y otro con argumentos. A mí lo que me gustaría es poder llamar al mismo, y decidir si quiero pasarle argumento o no.

Como ya conozco la programación funcional, puedo usar una de sus herramientas más poderosas en Python. La función [partial](https://docs.python.org/3/library/functools.html#functools.partial)

In [14]:
def debug(func=None, *, prefix=''):
    '''
    Decorator with or without optional arguments
    '''
    if func is None:
        return partial(debug, prefix=prefix)
    
    msg = prefix + func.__qualname__
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(msg)
        return func(*args, **kwargs)
    return wrapper

Ahora puedo llamar a debug sin argumentos, o con un argumento opcional

In [15]:
@debug(prefix="------")
def add(x, y):
    return x + y
@debug
def sub(x, y):
    return x - y
@debugargs(prefix="------")
def mul(x, y):
    return x * y
@debug
def div(x, y):
    return x / y


add(2, 2)
sub(8, 3)
mul(5,6)
div(16, 2)

------add
sub
------mul
div


8.0

Eso también hace feliz a Guido...

<img src="https://www.soldierx.com/system/files/hdb/guido-van-rossum.jpg">

Vayamos un poco más lejos... ¿Qué pasa si quiero decorar todos los métodos de una clase? Tengo que ir uno a uno. ¿Y si quisiera decorar a nivel de clase?

In [1]:
# Esta celda da el estilo al notebook
from IPython.core.display import HTML
css_file = '../static/styles/style.css'
HTML(open(css_file, "r").read())